In [ ]:
'''Train a model for the CY-metric'''

In [1]:
# Generate training data
import numpy as np
import os as os
import pickle as pickle
from cymetric.pointgen.pointgen import PointGenerator

monomials = 5*np.eye(5, dtype=np.int64)
coefficients = np.ones(5)
kmoduli = np.ones(1)
ambient = np.array([4])

pg = PointGenerator(monomials, coefficients, kmoduli, ambient)
points = pg.generate_points(100)
pg.cy_condition(points)

dirname = 'fermat_pg'
n_p = 100000
kappa = pg.prepare_dataset(n_p, dirname)
data = np.load(os.path.join(dirname, 'dataset.npz'))
pg.prepare_basis(dirname, kappa=kappa)

/Users/ed/miniforge3/envs/cymetric/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/ed/miniforge3/envs/cymetric/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
pointgen:INFO:Vol_k: 5.000000000000001, Vol_cy: 284.2561104432773.


0

In [4]:
import numpy as np
import os as os
import tensorflow as tf
tfk = tf.keras
from cymetric.models.tfhelper import prepare_tf_basis, train_model

data = np.load(os.path.join(dirname, 'dataset.npz'))
BASIS = np.load(os.path.join(dirname, 'basis.pickle'), allow_pickle=True)

In [5]:
#Notebook to generate the CY-metric model
#Setup the ML CY-metric model
from cymetric.models.callbacks import RicciCallback, SigmaCallback, VolkCallback, KaehlerCallback, TransitionCallback
from cymetric.models.tfmodels import MultFSModel
from cymetric.models.metrics import SigmaLoss, KaehlerLoss, TransitionLoss, VolkLoss, RicciLoss, TotalLoss

BASIS = prepare_tf_basis(BASIS)
kappa = np.real(BASIS['KAPPA'].numpy())

#Define callbacks --> needed?
rcb = RicciCallback((data['X_val'], data['y_val']), data['val_pullbacks'])
scb = SigmaCallback((data['X_val'], data['y_val']))
volkcb = VolkCallback((data['X_val'], data['y_val']))
kcb = KaehlerCallback((data['X_val'], data['y_val']))
tcb = TransitionCallback((data['X_val'], data['y_val']))
cb_list = [rcb, scb, kcb, tcb, volkcb]

#ML hyperparameters
nlayer = 3
nHidden = 64
act = 'gelu'
nEpochs = 1
bSizes = [64, 50000]
alpha = [1., 1., 1., 1., 1.]
nfold = 3
n_in = 2*5
n_out = nfold**2
cmetrics = [TotalLoss(), SigmaLoss(), KaehlerLoss(), TransitionLoss(), VolkLoss(), RicciLoss()]
opt = tfk.optimizers.Adam()

#Define the model
nn = tf.keras.Sequential()
nn.add(tfk.Input(shape=(n_in)))
for i in range(nlayer):
    nn.add(tfk.layers.Dense(nHidden, activation=act))
nn.add(tfk.layers.Dense(n_out, use_bias=False))

fmodel = MultFSModel(nn, BASIS, alpha=alpha)

#Train the model
fmodel, training_history = train_model(fmodel, data, optimizer=opt, epochs=nEpochs, batch_sizes=bSizes, 
                                       verbose=1, custom_metrics=cmetrics, callbacks=cb_list)

absl:WARNING:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.



Epoch  1/1


tensorflow:WARNING:You are casting an input of type complex64 to an incompatible dtype float32.  This will discard the imaginary part and may not be what you intended.
tensorflow:WARNING:You are casting an input of type complex64 to an incompatible dtype float32.  This will discard the imaginary part and may not be what you intended.
tensorflow:WARNING:You are casting an input of type complex64 to an incompatible dtype float32.  This will discard the imaginary part and may not be what you intended.
tensorflow:WARNING:You are casting an input of type complex64 to an incompatible dtype float32.  This will discard the imaginary part and may not be what you intended.
tensorflow:WARNING:You are casting an input of type complex64 to an incompatible dtype float32.  This will discard the imaginary part and may not be what you intended.
tensorflow:WARNING:You are casting an input of type complex64 to an incompatible dtype float32.  This will discard the imaginary part and may not be what you in

2/2 [==============================] - ETA: 0s - loss: 0.1113 - sigma_loss: 0.0941 - kaehler_loss: 0.0066 - transition_loss: 0.0107 - volk_loss: 0.0000e+00 - ricci_loss: 0.0000e+00

tensorflow:WARNING:You are casting an input of type complex64 to an incompatible dtype float32.  This will discard the imaginary part and may not be what you intended.


 - Ricci measure val:      1.2842
 - Sigma measure val:      0.1028
 - Kaehler measure val:    0.0065
 - Transition measure val: 0.0107
 - Volk val:               4.9861
2/2 [==============================] - 14s 8s/step - loss: 0.1113 - sigma_loss: 0.0941 - kaehler_loss: 0.0066 - transition_loss: 0.0107 - volk_loss: 0.0000e+00 - ricci_loss: 0.0000e+00 - ricci_val: 1.2842 - sigma_val: 0.1028 - kaehler_val: 0.0065 - transition_val: 0.0107 - volk_val: 4.9861


In [8]:
#Save the trained model
save_filepath = '/Users/ed/Documents/Projects/G2/Flows/G2Metric/github/cy_models/test_model.keras'
fmodel.save(save_filepath)

In [10]:
#Import the trained model
imported_model = tf.keras.models.load_model(save_filepath)
